<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/90_plane_crashes_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [ ]:
%%capture
!pip install --no-deps fiona GeodesicLinesToGIS

In [ ]:
import geopandas as gpd
import pooch
from geodesiclinestogis.geodesicline2gisfile import GeodesicLine2Gisfile
from shapely import wkt
from shapely.geometry import LineString

In [ ]:
plane_crashes_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/refs/heads/main/vector/plane_crashes.gpkg',
    known_hash='bbfe8e7d32fcec699960f486fec3dfad69cebbcac2d223003749f41c907afad0'
)

In [ ]:
plane_crashes = (
    gpd.read_file(plane_crashes_file)
    .dropna(subset=['From_pt'])
    .dropna(subset=['To_pt'])
    .assign(
        From_pt=lambda df_: df_.From_pt.apply(wkt.loads),
        To_pt=lambda df_: df_.To_pt.apply(wkt.loads),
    )
    .query('From_pt != To_pt')
    .reset_index(drop=True)
)

In [ ]:
def get_geodesic_line(pts):
    try:
        gtg = GeodesicLine2Gisfile()
        cd = gtg.gdlComp(pts, km_pts=5)
        return LineString(cd)
    except:
        return None


for ix, rec in plane_crashes.iterrows():
    pts = [*rec.From_pt.coords[0]] + [*rec.To_pt.coords[0]]
    plane_crashes.loc[ix, 'geometry'] = get_geodesic_line(pts)

plane_crashes = (
    gpd.GeoDataFrame(plane_crashes)
    .dropna(subset=['geometry'])
    .set_crs('EPSG:4326')
    .assign(length=lambda df_: df_.geometry.length)
    # .query('Aboard_Count > 50')
    # .query('80 < length < 100')
)

ERROR:root:Error: inv_intermediate: npts and del_s are mutually exclusive, only one of them must be != 0.
ERROR:root:Error: inv_intermediate: npts and del_s are mutually exclusive, only one of them must be != 0.
ERROR:root:Error: inv_intermediate: npts and del_s are mutually exclusive, only one of them must be != 0.
ERROR:root:Error: inv_intermediate: npts and del_s are mutually exclusive, only one of them must be != 0.
<ipython-input-5-8c2e13bfe3c3>:18: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  .assign(length=lambda df_: df_.geometry.length)


In [ ]:
plane_crashes.drop(columns=['From_pt', 'To_pt']).to_file('crash_lines.gpkg')